In [1]:
import os

In [2]:
import cv2
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from methods import brenner_gradient, laplacian, sobel_variance, tenengrad
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
FEAT_COLUMNS = [f.__name__ for f in [brenner_gradient, laplacian, sobel_variance, tenengrad]]

In [4]:
def extract_frames(video_path: str, output_folder: str, frame_interval: int = 1):
    cap = cv2.VideoCapture(f"{video_path}")
    os.makedirs(f"{output_folder}", exist_ok=True)
    frame_count = 0
    saved_count = 0
    if cap.isOpened():
        print("Video file successfully retrieved\n")
    else:
        print("Video file wasn't retrieved properly\n")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_interval == 0:
            cv2.imwrite(f"{output_folder}/{saved_count:04d}.jpg", frame)
            saved_count += 1
        frame_count += 1
    cap.release()
    print(f"Extracted {saved_count}/{frame_count} frames to {output_folder}\n")

In [5]:
def compute_features(image: np.ndarray):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    return [
        brenner_gradient(gray),
        sobel_variance(gray),
        tenengrad(gray),
        laplacian(gray),
    ]

In [6]:
def create_dataframe(image_folder: str):
    features = []
    files = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]
    files.sort()
    for file in files:
        image = cv2.imread(f"{image_folder}/{file}")
        features.append([file] + compute_features(image))
    return pd.DataFrame(features, columns=["file"] + FEAT_COLUMNS)

In [7]:
def label_dataframe(df: pd.DataFrame, image_folder: str):
    files = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]
    df["label"] = (df["file"].isin(files)).astype(int)
    return df

In [8]:
def train_models(X_train, y_train):
    models = {
        "Logistic Regression": LogisticRegression(),
        "KNN": KNeighborsClassifier(),
        "Decision Tree": DecisionTreeClassifier(),
        "CatBoost": CatBoostClassifier(verbose=0),
    }
    for name, model in models.items():
        model.fit(X_train, y_train)
    return models

In [9]:
def evaluate(models: dict, X_test, y_test):
    results = []
    for name, model in models.items():
        y_pred = model.predict(X_test)
        results.append(
            {"Model": name, "Precision": precision_score(y_test, y_pred), "Recall": recall_score(y_test, y_pred)}
        )
    return pd.DataFrame(results)

In [10]:
def predict_image(image_path: str, models: dict):
    results = []
    image = cv2.imread(image_path)
    features = pd.DataFrame([compute_features(image)], columns=FEAT_COLUMNS)
    for name, model in models.items():
        results.append({"Model": name, "Prediction": model.predict(features)[0]})
    return pd.DataFrame(results)

In [13]:
video_path = "../autofocus1.mp4"
all_folder = "../frames/all/"
focus_folder = "../frames/focus/"
frame_interval = 7
pd.set_option("display.max_rows", None)

In [42]:
# Extract-label
extract_frames(video_path, all_folder, frame_interval)
df = create_dataframe(all_folder)
df = label_dataframe(df, focus_folder)
print(f"Data\n{df}\n")

Video file successfully retrieved

Extracted 80/560 frames to ../frames/all/

Data
        file  brenner_gradient    laplacian  sobel_variance  tenengrad  label
0   0000.jpg           4047854  6488.211311        1.855450   0.755885      0
1   0001.jpg           4686576  6511.141308        1.959006   0.795287      0
2   0002.jpg           4083646  6496.856189        1.866986   0.786307      0
3   0003.jpg           3471114  6430.889991        1.756883   0.801344      0
4   0004.jpg           6984812  6563.810571        2.185118   0.804965      0
5   0005.jpg           4442862  6507.917825        1.932746   0.793810      0
6   0006.jpg           3797936  6470.765084        1.831901   0.803371      0
7   0007.jpg           3930116  6473.634143        1.842598   0.790361      0
8   0008.jpg           4990320  6522.070613        2.004973   0.808378      0
9   0009.jpg           3685554  6459.222045        1.801407   0.800293      0
10  0010.jpg           5172094  6524.988100        2.021082

In [47]:
# Split
X = df[FEAT_COLUMNS]
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [48]:
# Train-evaluate
models = train_models(X_train, y_train)
results = evaluate(models, X_test, y_test)
print(f"Train results\n{results}\n")

Train results
                 Model  Precision    Recall
0  Logistic Regression        1.0  0.833333
1                  KNN        1.0  0.833333
2        Decision Tree        1.0  0.833333
3             CatBoost        1.0  0.833333



In [49]:
# Predict
results = predict_image("../maxim-bogdanov-wjAR4jo979Y-unsplash.jpg", models)
print(f"Prediction\n{results}\n")

Prediction
                 Model  Prediction
0  Logistic Regression           1
1                  KNN           1
2        Decision Tree           0
3             CatBoost           1

